# Wide and Deep on TensorFlow (notebook style)

Copyright 2016 Google Inc. All Rights Reserved. Licensed under the Apache License, Version 2.0 (the "License"); you may not use this file except in compliance with the License. You may obtain a copy of the License at
 http://www.apache.org/licenses/LICENSE-2.0

Unless required by applicable law or agreed to in writing, software distributed under the License is distributed on an "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied. See the License for the specific language governing permissions and limitations under the License.

# Introduction

This notebook uses the tf.learn API in TensorFlow to answer a yes/no question. This is called a binary classification problem: Given census data about a person such as age, gender, education and occupation (the features), we will try to predict whether or not the person earns more than 50,000 dollars a year (the target label). 

Given an individual's information our model will output a number between 0 and 1, which can be interpreted as the model's certainty that the individual has an annual income of over 50,000 dollars, (1=True, 0=False)


# Imports and constants
First we'll import our libraries and set up some strings for column names. We also print out the version of TensorFlow we are running.

In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import time

import numpy as np
import tensorflow as tf

tf.logging.set_verbosity(tf.logging.ERROR) 
# Set to INFO for tracking training, default is WARN 

print("Using TensorFlow version %s" % (tf.__version__))

CATEGORICAL_COLUMNS = ["workclass", "education", 
                       "marital_status", "occupation", 
                       "relationship", "race", 
                       "gender", "native_country"]

# Columns of the input csv file
COLUMNS = ["age", "workclass", "fnlwgt", "education", 
           "education_num", "marital_status",
           "occupation", "relationship", "race", 
           "gender", "capital_gain", "capital_loss",
           "hours_per_week", "native_country", "income_bracket"]

# Feature columns for input into the model
FEATURE_COLUMNS = ["age", "workclass", "education", 
                   "education_num", "marital_status",
                   "occupation", "relationship", "race", 
                   "gender", "capital_gain", "capital_loss",
                   "hours_per_week", "native_country"]

Using TensorFlow version 1.2.0


# Pandas data exploration
We load the data into pandas because it is small enough to manage in memory, and look at some properties.

In [2]:
import pandas as pd

df = pd.read_csv("adult.data.csv", header=None, names=COLUMNS)

In [16]:
df.head()

,age,workclass,fnlwgt,education,education_num,marital_status,occupation,relationship,race,gender,capital_gain,capital_loss,hours_per_week,native_country,income_bracket
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K


In [4]:
df.describe(include=[np.number])

,age,fnlwgt,education_num,capital_gain,capital_loss,hours_per_week
count,32561.000000,3.256100e+04,32561.000000,32561.000000,32561.000000,32561.000000
mean,38.581647,1.897784e+05,10.080679,1077.648844,87.303830,40.437456
std,13.640433,1.055500e+05,2.572720,7385.292085,402.960219,12.347429
min,17.000000,1.228500e+04,1.000000,0.000000,0.000000,1.000000
25%,28.000000,1.178270e+05,9.000000,0.000000,0.000000,40.000000
50%,37.000000,1.783560e+05,10.000000,0.000000,0.000000,40.000000
75%,48.000000,2.370510e+05,12.000000,0.000000,0.000000,45.000000
max,90.000000,1.484705e+06,16.000000,99999.000000,4356.000000,99.000000


In [5]:
df.describe(include=[np.object])

,workclass,education,marital_status,occupation,relationship,race,gender,native_country,income_bracket
count,32561,32561,32561,32561,32561,32561,32561,32561,32561
unique,9,16,7,15,6,5,2,42,2
top,Private,HS-grad,Married-civ-spouse,Prof-specialty,Husband,White,Male,United-States,<=50K
freq,22696,10501,14976,4140,13193,27816,21790,29170,24720


In [5]:
df.corr()

,age,fnlwgt,education_num,capital_gain,capital_loss,hours_per_week
age,1.000000,-0.076646,0.036527,0.077674,0.057775,0.068756
fnlwgt,-0.076646,1.000000,-0.043195,0.000432,-0.010252,-0.018768
education_num,0.036527,-0.043195,1.000000,0.122630,0.079923,0.148123
capital_gain,0.077674,0.000432,0.122630,1.000000,-0.031615,0.078409
capital_loss,0.057775,-0.010252,0.079923,-0.031615,1.000000,0.054256
hours_per_week,0.068756,-0.018768,0.148123,0.078409,0.054256,1.000000


# Input file parsing

This section puts the file into a `Reader` which reads from the file one batch at a time. 

We set up the Tensors to be a dictionary of features mapping from their string name to the tensor value.

Note that the `_input_fn()` function is wrapped, enabling it to be used for different files.

NOTE: This reads from the input file directly via TensorFlow, rather than using an intermediate tool such as pandas to load the entire dataset into memory first. This is done to enable the system to scale to large inputs.

## More about input functions

The input function is how we will feed the input data into the model during training and evaluation. 
The structure that must be returned is a pair, where the first element is a dict of the column names (features) mapped to a tensor of values, and the 2nd element is a tensor of values representing the answers (labels). Recall that a tensor is just a general term for an n-dimensional array.

This could be represented as: `map(column_name => [Tensor of values]) , [Tensor of labels])`

More concretely, for this particular dataset, something like this:

    { 
      'age':            [ 39, 50, 38, 53, 28, … ], 
      'marital_status': [ 'Married-civ-spouse', 'Never-married', 'Widowed', 'Widowed' … ],
       ...
      'gender':           ['Male', 'Female', 'Male', 'Male', 'Female',, … ], 
    } , 
    [ 1, 1, 0, 1, 1, 1, 0, 0, 0, 0, 1, 1]
    
Additionally, we define which columns of the input data we will treat as categorical vs continuous, using the global `CATEGORICAL_COLUMNS`.

You can try different values for `BATCH_SIZE` to see how they impact your results

In [19]:
BATCH_SIZE = 40

def generate_input_fn(filename, batch_size=BATCH_SIZE):
    def _input_fn():
        filename_queue = tf.train.string_input_producer([filename])
        reader = tf.TextLineReader()
        # Reads out batch_size number of lines
        key, rows = reader.read_up_to(filename_queue, 
                                       num_records=batch_size)

        # record_defaults should match the datatypes of each respective column.
        record_defaults = [[0], [" "], [0], [" "], [0],
                           [" "], [" "], [" "], [" "], [" "],
                           [0], [0], [0], [" "], [" "]]

#         rows = tf.Print(rows, [tf.shape(rows)], 'rows before expanding')

        # changes from shape = [Batch_size] to shape = [Batch_size, 1]
        rows = tf.expand_dims(rows, axis=-1)
#         rows = rows[:, np.newaxis]

#         rows = tf.Print(rows, [tf.shape(rows)], 'rows after expanding')
        
        # Decode CSV data that was just read out. 
        columns = tf.decode_csv(
            rows, record_defaults=record_defaults)

        # features is a dictionary that maps from column names to tensors of the data.
        # income_bracket is the last column of the data. Note that this is NOT a dict.
        all_columns = dict(zip(COLUMNS, columns))
        
        # Save the income_bracket column as our labels 
        # dict.pop() returns the popped array of income_bracket values
        income_bracket = all_columns.pop('income_bracket')

        # remove the fnlwgt key, which is not used
        all_columns.pop('fnlwgt', 'fnlwgt key not found')

        # the remaining columns are our features
        features = all_columns

        # Convert ">50K" to 1, and "<=50K" to 0
        labels = tf.to_int32(tf.equal(income_bracket, " >50K"))

        return features, labels

    return _input_fn

print('input function configured')

input function configured


# Create Feature Columns
This section configures the model with the information about the model. There are many parameters here to experiment with to see how they affect the accuracy.

This is the bulk of the time and energy that is often spent on making a machine learning model work, called *feature selection* or *feature engineering*. We choose the features (columns) we will use for training, and apply any additional transformations to them as needed. 

### Sparse Columns
First we build the sparse columns.

Use `sparse_column_with_keys()` for columns that we know all possible values for.

Use `sparse_column_with_hash_bucket()` for columns that we want the the library to automatically map values for us.

In [18]:
# The layers module contains many utilities for creating feature columns.
from tensorflow.contrib import layers

# Sparse base columns.
gender = layers.sparse_column_with_keys(column_name="gender",
                                        keys=["female", "male"])
race = layers.sparse_column_with_keys(column_name="race",
                                      keys=["Amer-Indian-Eskimo",
                                            "Asian-Pac-Islander",
                                            "Black", "Other",
                                            "White"])

education = layers.sparse_column_with_hash_bucket(
  "education", hash_bucket_size=1000)
marital_status = layers.sparse_column_with_hash_bucket(
  "marital_status", hash_bucket_size=100)
relationship = layers.sparse_column_with_hash_bucket(
  "relationship", hash_bucket_size=100)
workclass = layers.sparse_column_with_hash_bucket(
  "workclass", hash_bucket_size=100)
occupation = layers.sparse_column_with_hash_bucket(
  "occupation", hash_bucket_size=1000)
native_country = layers.sparse_column_with_hash_bucket(
  "native_country", hash_bucket_size=1000)

print('Sparse columns configured')

Sparse columns configured


### Continuous columns
Second, configure the real-valued columns using `real_valued_column()`. 

In [20]:
# Continuous base columns.
age = layers.real_valued_column("age")
education_num = layers.real_valued_column("education_num")
capital_gain = layers.real_valued_column("capital_gain")
capital_loss = layers.real_valued_column("capital_loss")
hours_per_week = layers.real_valued_column("hours_per_week")

print('continuous columns configured')

continuous columns configured


### Transformations
Now for the interesting stuff. We will employ a couple of techniques to get even more out of the data.
 
* **bucketizing** turns what would have otherwise been a continuous feature into a categorical one. 
* **feature crossing** allows us to compute a model weight for specific pairings across columns, rather than learning them as independently. This essentially encodes related columns together, for situations where having 2 (or more) columns being certain values is meaningful. 

Only categorical features can be crossed. This is one reason why age has been bucketized.

For example, crossing education and occupation would enable the model to learn about: 

    education="Bachelors" AND occupation="Exec-managerial"

or perhaps 

    education="Bachelors" AND occupation="Craft-repair"

We do a few combined features (feature crosses) here. 

Add your own, based on your intuitions about the dataset, to try to improve on the model!

In [21]:
# Transformations.
age_buckets = layers.bucketized_column(
    age, boundaries=[ 18, 25, 30, 35, 40, 45, 50, 55, 60, 65 ])

education_occupation = layers.crossed_column(
    [education, occupation], hash_bucket_size=int(1e4))

age_race_occupation = layers.crossed_column(
    [age_buckets, race, occupation], hash_bucket_size=int(1e6))

country_occupation = layers.crossed_column(
    [native_country, occupation], hash_bucket_size=int(1e4))

print('Transformations complete')

Transformations complete


### Group feature columns into 2 objects

The wide columns are the sparse, categorical columns that we specified, as well as our hashed, bucket, and feature crossed columns. 

The deep columns are composed of embedded categorical columns along with the continuous real-valued columns. **Column embeddings** transform a sparse, categorical tensor into a low-dimensional and dense real-valued vector. The embedding values are also trained along with the rest of the model. For more information about embeddings, see the TensorFlow tutorial on [Vector Representations Words](https://www.tensorflow.org/tutorials/word2vec/), or [Word Embedding](https://en.wikipedia.org/wiki/Word_embedding) on Wikipedia.

The higher the dimension of the embedding is, the more degrees of freedom the model will have to learn the representations of the features. We are starting with an 8-dimension embedding for simplicity, but later you can come back and increase the dimensionality if you wish.



In [22]:
# Wide columns and deep columns.
wide_columns = [gender, race, native_country,
      education, occupation, workclass,
      marital_status, relationship,
      age_buckets, education_occupation,
      age_race_occupation, country_occupation]

deep_columns = [
  layers.embedding_column(workclass, dimension=8),
  layers.embedding_column(education, dimension=8),
  layers.embedding_column(marital_status, dimension=8),
  layers.embedding_column(gender, dimension=8),
  layers.embedding_column(relationship, dimension=8),
  layers.embedding_column(race, dimension=8),
  layers.embedding_column(native_country, dimension=8),
  layers.embedding_column(occupation, dimension=8),
  age,
  education_num,
  capital_gain,
  capital_loss,
  hours_per_week,
]

print('wide and deep columns configured')

wide and deep columns configured


# Create the model

You can train either a "wide" model, a "deep" model, or a "wide and deep" model, using the classifiers below. Try each one and see what kind of results you get.

* **Wide**: Linear Classifier
* **Deep**: Deep Neural Net Classifier
* **Wide & Deep**: Combined Linear and Deep Classifier

The `hidden_units` or `dnn_hidden_units` argument is to specify the size of each layer of the deep portion of the network. For example, `[12, 20, 15]` would create a network with the first layer of size 12, the second layer of size 20, and a third layer of size 15.

In [23]:
from tensorflow.contrib import learn

def create_model_dir(model_type):
    return 'models/model_' + model_type + '_' + str(int(time.time()))

# If new_model=False, pass in the desired model_dir 
def get_model(model_type, new_model=False, model_dir=None):
    if new_model or model_dir is None:
        model_dir = create_model_dir(model_type) # Comment out this line to continue training a existing model
    print("Model directory = %s" % model_dir)
    
    m = None
    
    # Linear Classifier
    if model_type == 'WIDE':
        m = learn.LinearClassifier(
            model_dir=model_dir, 
            feature_columns=wide_columns)

    # Deep Neural Net Classifier
    if model_type == 'DEEP':
        m = learn.DNNClassifier(
            model_dir=model_dir,
            feature_columns=deep_columns,
            hidden_units=[100, 50])

    # Combined Linear and Deep Classifier
    if model_type == 'WIDE_AND_DEEP':
        m = learn.DNNLinearCombinedClassifier(
                model_dir=model_dir,
                linear_feature_columns=wide_columns,
                dnn_feature_columns=deep_columns,
                dnn_hidden_units=[100, 70, 50, 25])
        
    print('estimator built')
    
    return m, model_dir
    
MODEL_TYPE = 'WIDE_AND_DEEP'
model_dir = create_model_dir(model_type=MODEL_TYPE)
m, model_dir = get_model(model_type = MODEL_TYPE, model_dir=model_dir)

Model directory = models/model_WIDE_AND_DEEP_1497568810
estimator built


# Fit the model (train it)

Run `fit()` to train the model. You can experiment with the `train_steps` and `BATCH_SIZE` parameters.

This can take some time, depending on the values chosen for `train_steps` and `BATCH_SIZE`.

Our datafile is hosted on Google Cloud Storage; the reader we created at the beginning knows how to read from it.

If you don't want to download a new copy of the dataset each time your script runs, you can download it locally using 

    gsutil cp gs://cloudml-public/census/data/adult.data.csv .
    gsutil cp gs://cloudml-public/census/data/adult.test.csv .

In [24]:
%%time 

train_file = str("adult.data.csv") 
# "gs://cloudml-public/census/data/adult.data.csv"
# storage.googleapis.com/cloudml-public/census/data/adult.data.csv

m.fit(input_fn=generate_input_fn(train_file, BATCH_SIZE), 
      steps=1000)

print('fit done')

fit done
CPU times: user 1min 8s, sys: 5.45 s, total: 1min 14s
Wall time: 1min 22s


# Evaluate the accuracy of the model
Let's see how the model did. We will evaluate all the test data.

In [25]:
test_file  = str("adult.test.csv") 
# "gs://cloudml-public/census/data/adult.test.csv"
# storage.googleapis.com/cloudml-public/census/data/adult.test.csv

results = m.evaluate(input_fn=generate_input_fn(test_file), 
                     steps=200)
print('evaluate done')
print(results)
print('Accuracy: %s' % results['accuracy'])

evaluate done
{'accuracy/baseline_label_mean': 0.0, 'loss': 0.693712, 'auc': 0.99999994, 'global_step': 1001, 'accuracy/threshold_0.500000_mean': 0.837125, 'recall/positive_threshold_0.500000_mean': 0.0, 'labels/prediction_mean': 0.21906112, 'accuracy': 0.837125, 'precision/positive_threshold_0.500000_mean': 0.0, 'labels/actual_label_mean': 0.0}
Accuracy: 0.837125


# Export a model optimized for inference
We can upload our trained model to the Cloud Machine Learning Engine's Prediction Service, which will take care of serving our model and scaling it. The code below exports our trained model to a `saved_model.pb` file and a `variables` folder where the trained weights are stored. 

The `export_savedmodel()` function expects a `serving_input_fn()`, which returns the mapping from the data that the Prediction Service passes in to the data that should be fed into the trained TensorFlow prediction graph.

In [27]:
from tensorflow.contrib.learn.python.learn.utils import input_fn_utils

def column_to_dtype(column):
    if column in CATEGORICAL_COLUMNS:
        return tf.string
    else:
        return tf.float32

def serving_input_fn():
    feature_placeholders = {
        column: tf.placeholder(column_to_dtype(column), [None])
        for column in FEATURE_COLUMNS
    }
    # DNNCombinedLinearClassifier expects rank 2 Tensors, but inputs should be
    # rank 1, so that we can provide scalars to the server
    features = {
        key: tensor[:, np.newaxis] # tf.expand_dims(tensor, axis=-1)
        for key, tensor in feature_placeholders.items()
    }
    
    return input_fn_utils.InputFnOps(
        features, # input into graph
        None,
        feature_placeholders # tensor input converted from request 
    )

In [28]:
export_folder = m.export_savedmodel(
    export_dir_base = model_dir + '/export',
    serving_input_fn=serving_input_fn
)

print('model exported successfully to {}'.format(export_folder))

model exported successfully to models/model_WIDE_AND_DEEP_1497568810/export/1497569315


# Using Experiments to manage the training workflow
TensorFlow also offers an "Experiments" framework to help manage your training for you. Using it will take away some of the boilerplate and automate certain actions, but it's important to understand what is happening (what we did above) before using something like this.

Notice that we don't need to create additional wrapper functions here, but rather, we can directly use the various functions we already defined previously. 

In [16]:
from tensorflow.contrib.learn.python.learn import learn_runner
from tensorflow.contrib.learn.python.learn.utils import saved_model_export_utils

# output_dir is an arg passed in by the learn_runner.run() call.
def experiment_fn(output_dir):
    
    print(output_dir)
    
    train_input_fn = generate_input_fn(train_file, BATCH_SIZE)
    eval_input_fn = generate_input_fn(test_file)
    my_model, model_dir = get_model(model_type=MODEL_TYPE, 
                  model_dir=output_dir)

    experiment = tf.contrib.learn.Experiment(
        my_model,
        train_input_fn=train_input_fn,
        eval_input_fn=eval_input_fn,
        train_steps=1000
        ,
        export_strategies=[saved_model_export_utils.make_export_strategy(
            serving_input_fn,
            default_output_alternative_key=None,
            exports_to_keep=1
        )]
    )
    return experiment


# Run the experiment

model_dir=create_model_dir(model_type=MODEL_TYPE)
metrics, output_folder = learn_runner.run(experiment_fn, model_dir)

print('Accuracy: {}'.format(metrics['accuracy']))
print('Model exported to {}'.format(output_folder))

models/model_WIDE_AND_DEEP_1496942828
Model directory = models/model_WIDE_AND_DEEP_1496942828
estimator built
Accuracy: 0.820999979973
Model exported to ['models/model_WIDE_AND_DEEP_1496942828/export/Servo/1496942952']


# Conclusions

In this Juypter notebook, we have configured, created, and evaluated a Wide & Deep machine learning model, that combines the powers of a Linear Classifier with a Deep Neural Network, using TensorFlow's tf.learn module.

With this working example in your toolbelt, you are ready to explore the wide (and deep) world of machine learning with TensorFlow! Some ideas to help you get going:
* Change the features we used today. Which columns do you think are correlated and should be crossed? Which ones do you think are just adding noise and could be removed to clean up the model?
* Swap in an entirely new dataset! There are many dataset available on the web, or use a dataset you possess! Check out https://archive.ics.uci.edu/ml to find your own dataset. 